In [1]:
import numpy as np
import glob

In [2]:
def get_SNR(file):
    with open(file, "rt")as f:
        a = f.readlines()
    lst = []
    for x in a:
        if "Value:" in x:
            lst.append(float(x.split("Value: ")[1]))
        #print(x.split("bla"))
    lst = lst[len(lst)//2 : ]
    return (np.array(lst).mean(), np.array(lst).std())

In [3]:
def get_BER(file):
    with open(file, "rb")as f:
        _ = f.read()
    
    rcv = np.frombuffer(_, dtype = np.uint8)
    aux = np.unpackbits(rcv)
    
    tx = np.array([0, 255, 40, 250, 37, 42, 186], dtype = np.uint8)
    
    pos = np.argmax(np.correlate(np.unpackbits(rcv), np.unpackbits(tx)))
    rcv = np.packbits(np.roll(np.unpackbits(rcv), -pos))
    tx_extended = np.tile(tx, int(rcv.shape[0]/len(tx)))
    
    return (1 - np.mean(np.equal(np.unpackbits(rcv), np.unpackbits(tx_extended))))

In [4]:
get_BER("rcv_0.data")

0.03976392857142852

In [5]:
get_SNR("rcv_0.txt")

(3.468054047619048, 0.4091092542810278)

In [6]:
glob.glob("*.txt")

['rcv_4.txt', 'rcv_2.txt', 'rcv_6.txt', 'rcv_1.txt', 'rcv_3.txt', 'rcv_0.txt']

In [7]:
dc = {}

In [8]:
for file in glob.glob("*.txt"):
    mean, std = get_SNR(file)
    dc[file] = (get_SNR(file), get_BER(file.split(".")[0] + ".data"))
    print(file)
    #print(f"{file} (SNR Mean, SNR Std): {mean}, {std}")

rcv_4.txt
rcv_2.txt
rcv_6.txt
rcv_1.txt
rcv_3.txt
rcv_0.txt


In [9]:
sorted(dc.items(), key = lambda x: x[1][0])

[('rcv_2.txt',
  ((1.5815411190476192, 0.4853298958173268), 0.11585839285714283)),
 ('rcv_0.txt', ((3.468054047619048, 0.4091092542810278), 0.03976392857142852)),
 ('rcv_3.txt',
  ((4.559230476190477, 0.3337094797442857), 0.019170714285714263)),
 ('rcv_4.txt',
  ((5.434059523809524, 0.3332578299794776), 0.0063592857142856785)),
 ('rcv_1.txt',
  ((5.4509823809523805, 0.3027723468203346), 0.006938571428571461)),
 ('rcv_6.txt',
  ((7.716105476190478, 0.2748177624501492), 0.0005371428571429027))]

In [10]:
lst_SNR = []
lst_BER = []

for x in sorted(dc.items(), key = lambda x: x[1][0]):
    print(f"{x[0]}:   SNR Mean, SNR Std: {x[1][0]}, BER: {x[1][1]}")
    lst_SNR.append(x[1][0][0])
    lst_BER.append(x[1][1])

rcv_2.txt:   SNR Mean, SNR Std: (1.5815411190476192, 0.4853298958173268), BER: 0.11585839285714283
rcv_0.txt:   SNR Mean, SNR Std: (3.468054047619048, 0.4091092542810278), BER: 0.03976392857142852
rcv_3.txt:   SNR Mean, SNR Std: (4.559230476190477, 0.3337094797442857), BER: 0.019170714285714263
rcv_4.txt:   SNR Mean, SNR Std: (5.434059523809524, 0.3332578299794776), BER: 0.0063592857142856785
rcv_1.txt:   SNR Mean, SNR Std: (5.4509823809523805, 0.3027723468203346), BER: 0.006938571428571461
rcv_6.txt:   SNR Mean, SNR Std: (7.716105476190478, 0.2748177624501492), BER: 0.0005371428571429027


In [11]:
lst_BER

[0.11585839285714283,
 0.03976392857142852,
 0.019170714285714263,
 0.0063592857142856785,
 0.006938571428571461,
 0.0005371428571429027]

In [12]:
lst_SNR

[1.5815411190476192,
 3.468054047619048,
 4.559230476190477,
 5.434059523809524,
 5.4509823809523805,
 7.716105476190478]